In [36]:
import yfinance as yf
import pandas_datareader as pdr

start="2020-01-01"
end="2025-08-25"

In [37]:
data = yf.download("AMZN, HD, COST, BURL, ULTA, WMT, BBY, TGT",
                    start=start,
                    end=end) 

/tmp/ipykernel_11073/1703306638.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("AMZN, HD, COST, BURL, ULTA, WMT, BBY, TGT",
[*********************100%***********************]  8 of 8 completed


In [38]:
# data.head()
# drop all data except the opening and volume columns.
data = data[['Open', 'Volume']].copy()
# data.head()

# refactor input so that the two columns are just Open and Volume, with the specific 
# ticker name gone; that is, all the tickers should be aggregatted under the 
# Open and Volume columns.
data = data.stack(level='Ticker', future_stack=True).reset_index()

# add the month as a number from 1 to 12 as a column to the dataframe
data['Month'] = data['Date'].dt.month

treasury_data = pdr.get_data_fred('GS10', start=start, end=end)

# Reset index to make DATE a column for merging
treasury_data = treasury_data.reset_index()

# Treasury data is monthly, stock data is daily
# We need to create a year-month column for both datasets to merge on
treasury_data['Year_Month'] = treasury_data['DATE'].dt.to_period('M')
data['Year_Month'] = data['Date'].dt.to_period('M')

# Merge treasury data with stock data on Year_Month
# Using left join to keep all stock data rows
data = data.merge(treasury_data[['Year_Month', 'GS10']], on='Year_Month', how='left')

# Drop the temporary Year_Month column
data = data.drop('Year_Month', axis=1)

print(f"\nData shape after merge: {data.shape}")
print("Columns after merge:", data.columns.tolist())
data.info()
data.isnull().sum()


Data shape after merge: (11344, 6)
Columns after merge: ['Date', 'Ticker', 'Open', 'Volume', 'Month', 'GS10']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11344 entries, 0 to 11343
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    11344 non-null  datetime64[ns]
 1   Ticker  11344 non-null  object        
 2   Open    11344 non-null  float64       
 3   Volume  11344 non-null  int64         
 4   Month   11344 non-null  int32         
 5   GS10    11216 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(1), int64(1), object(1)
memory usage: 487.6+ KB


Date        0
Ticker      0
Open        0
Volume      0
Month       0
GS10      128
dtype: int64

In [39]:
# Isolate the rows where GS10 is null
nan_rows = data[data['GS10'].isnull()]

# Print the unique dates or months for those rows
print("Unique months with NaN values in GS10:")
print(nan_rows['Date'].dt.to_period('M').unique())

Unique months with NaN values in GS10:
<PeriodArray>
['2025-08']
Length: 1, dtype: period[M]
